<a href="https://colab.research.google.com/github/JamieBali/sudoku/blob/main/HNN_Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HNN Sudoku Solver

In [103]:
import numpy as np
import pandas as pd, csv
import math, random

In [2]:
trivial = [[8,9,1,2,7,0,5,6,3],[6,0,3,1,8,5,9,0,0],[4,5,7,6,3,9,0,0,2],[5,0,6,4,1,7,2,3,0],[7,0,2,9,0,3,8,1,6],[3,1,0,0,2,6,0,5,4],[9,3,8,5,4,0,6,7,0],[1,6,4,0,9,0,3,2,5],[0,7,5,3,6,1,4,9,8]]

# this is a very simple 4x4 sudoku from sudoku.com which we can use to test the network.
four = [[0,3,4,0],[4,0,0,2],[1,0,0,3],[0,2,1,0]]

# these three were taken randomly from sudoku.com
easy = [[0,0,0,0,7,9,0,3,0],[5,0,2,0,6,1,4,7,8],[3,7,6,0,8,5,9,0,2],[0,1,7,5,0,0,8,0,0],[2,0,9,8,3,0,0,0,0],[0,0,0,0,2,0,0,4,0],[0,0,0,0,5,0,2,0,1],[0,2,3,0,0,0,0,5,4],[1,0,0,7,0,0,0,0,0]]
medium = [[0,3,1,0,5,0,0,2,0],[0,0,0,0,0,2,9,0,5],[2,0,0,0,1,0,0,0,0],[3,5,0,0,9,0,0,7,0],[7,0,0,5,0,0,0,4,0],[0,1,0,7,0,3,2,0,0],[1,2,6,3,0,0,0,0,0],[0,9,0,8,0,5,0,0,0],[5,0,0,0,2,0,7,0,0]]
hard = [[0,4,0,0,0,5,0,6,0],[0,0,5,4,2,0,0,0,0],[0,0,1,6,0,3,5,0,4],[0,0,0,0,0,0,7,0,0],[0,3,7,0,0,0,0,1,0],[9,0,0,0,0,4,3,5,0],[0,0,4,2,5,0,0,0,0],[0,0,0,0,0,0,0,7,6],[6,0,9,0,7,0,0,0,5]]

# this is the solved version of the "easy" sudoku above.
solved = [[8,4,1,2,7,9,6,3,5],[5,9,2,3,6,1,4,7,8],[3,7,6,4,8,5,9,1,2],[4,1,7,5,9,6,8,2,3],[2,5,9,8,3,4,1,6,7],[6,3,8,1,2,7,5,4,9],[7,6,4,9,5,3,2,8,1],[9,2,3,6,1,8,7,5,4],[1,8,5,7,4,2,3,9,6]]

# Hopfield Neurones

We will construct each neurone in the network as its own object and create an array of them to act as the network. This will allow us to make edits to the functionality of the network easier.

In [59]:
class Neuron:
  input=0
  output=0
  energy=0
  i=0
  j=0
  k=0
  
  def __init__(self, i, j, k, cell_value, size): #constructor to initialize values for each neuron
    self.i = i
    self.j = j
    self.k = k
    self.input = cell_value
    self.size = size

  def step_neurone(self, output_for_each_neuron):    
    i = self.i
    j = self.j
    k = self.k
    first_term = self.input
    second_term = 0.0
    
    ###
    # Domain constraints - increase energy with violations to rules of sudoku
    ###
    for i_itt in range(0,self.size): # in the same row
      if(i_itt!=i):                  # but not self
        second_term += output_for_each_neuron[i_itt][j][k]

    for j_itt in range(0,self.size): # in the same column
      if(j_itt!=j):                  # but not self
         second_term += output_for_each_neuron[i][j_itt][k]

    for k_itt in range(0,self.size): # in the same tile
      if(k_itt!=k):                  # but not self
        second_term += output_for_each_neuron[i][j][k_itt]
    
    sub_size = int(math.sqrt(self.size))
    sub_num = (math.floor(i/sub_size) * sub_size) + (math.floor(j/sub_size))
    r_start = math.floor(sub_num / sub_size)  * sub_size
    c_start = (sub_num % sub_size) * sub_size

    # 0 | 3 | 6
    # ---------
    # 1 | 4 | 7
    # ---------
    # 2 | 5 | 8
    
    for i_itt in range(r_start,r_start+sub_size): # for each tile in the sub-grid
      for j_itt in range(c_start, c_start+sub_size):
        if not (i_itt==i and j_itt==j): # but not self
          second_term += output_for_each_neuron[i_itt][j_itt][k]
          
    self.energy = first_term + second_term    
    self.output = self.energy
    return self.energy
  
  def get_output(self):
    return self.output

# Network Construction


In [113]:
class hopfieldNetwork():
  def __init__(self, puzzle, size, verbose = False):
    self.size = size
    self.verbose = verbose
    self.puzzle = self.network_format(puzzle)
    self.neurones = self.generate_neurones()
    self.steps = sum(np.array([int(x == 0) for x in np.copy(puzzle).flatten()])) - 1

  def network_format(self, sudoku):
    ret_puzzle = np.zeros((self.size, self.size, self.size)) 
    for i in range(self.size):
      for j in range(self.size):
        if sudoku[i][j] > 0:
          ret_puzzle[i][j][sudoku[i][j]-1] = 1
    return ret_puzzle

  def readable_format(self):
    result = np.zeros((self.size, self.size))
    for i in range(self.size):
      for j in range(self.size):
        for k in range(self.size):
          if self.puzzle[i][j][k] == 1:
            result[i][j] = k + 1
    return result

  def generate_neurones(self):
    neurones = []
    for i in range(self.size):
      for j in range(self.size):
        for k in range(self.size):
          neurones.append(Neuron(i, j, k, self.puzzle[i][j][k], self.size))
    return neurones

  def get_index(self, row, column, number):
    return int((column * (self.size ** 2)) + (row * self.size) + number)

  def step(self):
    for _ in range(2):  # the first iteration fills the grid with possibilities, and the second drops the energy of ones it dislikes.

      outputs = np.zeros((self.size, self.size, self.size))       # stores the outputs from each neurone after each step

      for i in range(self.size):
        for j in range(self.size):
          for k in range(self.size):
            outputs[i][j][k] = self.neurones[self.get_index(i,j,k)].get_output()

      for itt in range(0, self.size**3):
        self.neurones[itt].step_neurone(outputs)    

    self.build_board()

  def build_board(self):
    k_to_set = []
    lowest_k = -1
    for i in range(self.size):
      for j in range(self.size):
        if not(1 in self.puzzle[i][j]):
          for k in range(self.size):
            temp = self.neurones[self.get_index(i,j,k)].get_output() 
            if (temp < lowest_k or lowest_k == -1):
              lowest_k = temp
              k_to_set = [i,j,k]
    self.puzzle[k_to_set[0]][k_to_set[1]][k_to_set[2]] = 1    
    if self.verbose:
      print("set " + str(k_to_set[0]) + "," + str(k_to_set[1]) + " as " + str(k_to_set[2] + 1))  

  def reset_network(self):
    self.neurones = self.generate_neurones()   

  def is_solved(self):
    check_puzzle = np.zeros((self.size, self.size))
    for x in range(self.size):
      for y in range(self.size):
        for z in range(self.size):
          if self.puzzle[x][y][z] == 1:
            check_puzzle[x][y] = z + 1
    sums = np.zeros(self.size)
    for x in range(self.size):
      for y in range(self.size):
        sums[int(check_puzzle[x][y]) - 1] += 1
    for x in sums:
      if x != self.size:
        return False
    return True

  def run(self):
    for x in range(self.steps):
      self.step()
      self.reset_network()
    return self.readable_format(), self.is_solved()

# Testing

In [116]:
net = hopfieldNetwork(easy, 9)
returned_puzzle, success = net.run()
if success:
  print("SUCCESS")
else:
  print(returned_puzzle)

SUCCESS


# Experiments

In [121]:
from google.colab import drive
drive.mount("gdrive")

import pandas as pd, csv
import time

filepath = "/content/gdrive/My Drive/ColabNotebooks/IP/sudoku.csv"
inp = pd.read_csv(filepath)

puzzles = inp["quizzes"][-500:]

successful = 0
puzz_no = 1
times = []
solvable = []
for p in puzzles:
  print(puzz_no)
  usable_puzzle = np.array([int(t) for t in p]).reshape((9,9))
  a = time.perf_counter()
  net = hopfieldNetwork(usable_puzzle, 9)
  _, solve = net.run()
  b = time.perf_counter()
  if solve:
    successful += 1
    times.append(b-a)
    solvable.append(p)
  puzz_no += 1

print("Accuracy  : "  + str(successful / 10) + "%")
print("Avg. Time : " + str(np.average(times)))
print("Max Time  : " + str(np.max(times)))

Accuracy  : 9.2%
Avg. Time : 1.1624517273478254
Max Time  : 1.4333317010003157
